<a href="https://colab.research.google.com/github/5quidL0rd/Unsloth-project/blob/main/Unsloth_Haiku_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Install Dependencies

%%capture
!pip install unsloth
!pip install kaggle

!pip install trl

!pip install peft

!pip install torch



!pip install --upgrade tensorflow

!pip install --upgrade transformers

!pip install --upgrade transformers peft datasets accelerate



In [ ]:
%%capture
# Skip restarting message in Colab
import sys; modules = list(sys.modules.keys())
for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None

!pip install unsloth vllm
!pip install --upgrade pillow
# If you are running this notebook on local, you need to install `diffusers` too
# !pip install diffusers
# Temporarily install a specific TRL nightly version
!pip install git+https://github.com/huggingface/trl.git@e95f9fb74a3c3647b86f251b7e230ec51c64b72b

In [ ]:
from unsloth import is_bfloat16_supported #type:ignore
import torch
max_seq_length = 256 # Can increase for longer reasoning traces


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "meta-llama/Llama-3.2-1B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank, #type:ignore
    gpu_memory_utilization = 0.6, # Reduce if out of memory
    device_map="auto",
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 8, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = 8,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)



NotImplementedError: Unsloth: No NVIDIA GPU found? Unsloth currently only supports GPUs!

In [ ]:
# Step 2: Upload the Kaggle Dataset Manually for google collaborate
from google.colab import files

# Upload the CSV file manually from your local system (unzip beforehand for convenience)
uploaded = files.upload()




Saving all_haiku.csv to all_haiku.csv


In [ ]:
import pandas as pd

# Load the CSV file, assuming the correct headers are in the second row (index 1)
df = pd.read_csv('/content/all_haiku.csv', header=1)

# Display the first few rows to confirm the columns are correct now (highly recommend kaggle datasets are unpredictable)
print(df.head())


   0    fishing boats              colors of       the rainbow  tempslibres  \
0  1  ash wednesday--    trying to remember           my dream  tempslibres   
1  2     snowy morn--    pouring another cup   of black coffee  tempslibres   
2  3     shortest day           flames dance       in the oven  tempslibres   
3  4             haze  half the horse hidden  behind the house  tempslibres   
4  5          low sun        the lady in red     on high heels  tempslibres   

            FISHINGBOATSCOLORSOFTHERAINBOW  
0      ASHWEDNESDAYTRYINGTOREMEMBERMYDREAM  
1  SNOWYMORNPOURINGANOTHERCUPOFBLACKCOFFEE  
2          SHORTESTDAYFLAMESDANCEINTHEOVEN  
3     HAZEHALFTHEHORSEHIDDENBEHINDTHEHOUSE  
4            LOWSUNTHELADYINREDONHIGHHEELS  


In [ ]:
# Check the first few rows of the dataframe to understand the structure
print(df.head())

# Step 5: Rename Columns and Remove Irrelevant Columns
# Manually rename the columns (based on the dataset structure)
# It seems the 'First Line' column is already named correctly, but 'fishing boats', 'colors of', etc. need to be renamed

df.rename(columns={
    '0': 'First Line',
    'fishing boats': 'Second Line',  # Renaming based on actual dataset
    'colors of': 'Third Line',  # Renaming based on actual dataset
}, inplace=True)

# Check the column names after renaming
print(df.columns)

# Remove any irrelevant columns (such as 'tempslibres' or 'FISHINGBOATSCOLORSOFTHERAINBOW')
df = df[['First Line', 'Second Line', 'Third Line']]

# Check the cleaned dataframe to make sure everything is correct
print(df.head())




   0    fishing boats              colors of       the rainbow  tempslibres  \
0  1  ash wednesday--    trying to remember           my dream  tempslibres   
1  2     snowy morn--    pouring another cup   of black coffee  tempslibres   
2  3     shortest day           flames dance       in the oven  tempslibres   
3  4             haze  half the horse hidden  behind the house  tempslibres   
4  5          low sun        the lady in red     on high heels  tempslibres   

            FISHINGBOATSCOLORSOFTHERAINBOW  
0      ASHWEDNESDAYTRYINGTOREMEMBERMYDREAM  
1  SNOWYMORNPOURINGANOTHERCUPOFBLACKCOFFEE  
2          SHORTESTDAYFLAMESDANCEINTHEOVEN  
3     HAZEHALFTHEHORSEHIDDENBEHINDTHEHOUSE  
4            LOWSUNTHELADYINREDONHIGHHEELS  
Index(['First Line', 'Second Line', 'Third Line', 'the rainbow', 'tempslibres',
       'FISHINGBOATSCOLORSOFTHERAINBOW'],
      dtype='object')
   First Line      Second Line             Third Line
0           1  ash wednesday--    trying to remember 
1  

In [ ]:
# Step 6: Prepare the Dataset (Text Preprocessing)
# Ensure all columns are strings before concatenating
df['First Line'] = df['First Line'].fillna('').astype(str)
df['Second Line'] = df['Second Line'].fillna('').astype(str)
df['Third Line'] = df['Third Line'].fillna('').astype(str)

# Combine the 'First Line', 'Second Line', and 'Third Line' to create a full haiku for each row
poems = df['First Line'] + "\n" + df['Second Line'] + "\n" + df['Third Line']

# Clean the text by stripping whitespace and removing any empty haikus
poems = poems[poems.str.strip().apply(lambda x: len(x) > 0)].tolist()

# Join all the haikus into one large string, separated by new lines
poems_text = "\n".join(poems)

# Check the first few poems to ensure it's working
print(poems[:5])



['1\nash wednesday--\ntrying to remember ', '2\nsnowy morn--\npouring another cup', '3\nshortest day\nflames dance', '4\nhaze\nhalf the horse hidden', '5\nlow sun\nthe lady in red']


In [ ]:
# Step 7: Prepare the Dataset for Fine-Tuning (Formatting Prompts)
# Define prompt format for model training
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

from transformers import AutoTokenizer #type: ignore

#tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")

# Formatting function for haiku dataset
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["First Line"]
    inputs = examples["Second Line"]
    outputs = examples["Third Line"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Add EOS_TOKEN to end of each prompt
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

# Apply the formatting to the dataset
from datasets import Dataset #type: ignore

# Ensure all values are strings and replace NaN with empty strings
df['First Line'] = df['First Line'].fillna('').astype(str)
df['Second Line'] = df['Second Line'].fillna('').astype(str)
df['Third Line'] = df['Third Line'].fillna('').astype(str)

# Create a dataset from the haiku text, ensuring all values are strings
dataset = Dataset.from_dict({
    "First Line": df['First Line'],
    "Second Line": df['Second Line'],
    "Third Line": df['Third Line'],
})

# Map the formatting function to the dataset
dataset = dataset.map(formatting_prompts_func, batched=True)

# Now the dataset is ready for fine-tuning


Map:   0%|          | 0/144122 [00:00<?, ? examples/s]

NameError: name 'EOS_TOKEN' is not defined

In [ ]:
import os
import torch
import gc
from unsloth import FastLanguageModel #type: ignore
from trl import SFTTrainer #type: ignore
from transformers import TrainingArguments
from peft import get_peft_model, LoraConfig, TaskType
from accelerate import Accelerator

from trl import GRPOConfig, GRPOTrainer #type: ignore
training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "paged_adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 6, # Decrease if out of memory
    max_prompt_length = 256,
    max_completion_length = 200,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 250,
    save_steps = 250,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

trainer_stats = trainer_train() #type: ignore



NotImplementedError: Unsloth: No NVIDIA GPU found? Unsloth currently only supports GPUs!